In [ ]:
import tensorflow as tf
from tensorflow.compat.v1 import graph_util
import numpy as np
from matplotlib import pyplot as plt
import time

from models import *
from mnist import MNIST  # this is the MNIST data manager that provides training/testing batches

In [ ]:
class ConvolutionalAutoencoder(object):
    """

    """
    def __init__(self):
        """
        build the graph
        """
        tf.reset_default_graph()
        
        # place holder of input data
        x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])  # [#batch, img_height, img_width, #channels]
#         code = tf.placeholder(tf.float32, shape=[None, 20])  # [#batch, features]
        # encode
        conv1 = Convolution2D([3, 3, 1, 32], activation=tf.nn.relu, scope='conv_1')(x)
        pool1 = MaxPooling(kernel_shape=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', scope='pool_1')(conv1)
        conv2 = Convolution2D([3, 3, 32, 32], activation=tf.nn.relu, scope='conv_2')(pool1)
        pool2 = MaxPooling(kernel_shape=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', scope='pool_2')(conv2)
        #卷据池化 pass 10000, training loss 778.7437744140625 trained:1000000, 2511.816041 sec
#         conv3 = Convolution2D([3, 3, 32, 32], activation=tf.nn.relu, scope='conv_3')(pool2)
#         pool3 = MaxPooling(kernel_shape=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', scope='pool_3')(conv3)
        #只卷积 pass 10000, training loss 586.9559326171875  trained:1000000, 1589.608651 sec
        conv3 = Convolution2D([3, 3, 32, 32], strides=[1, 2, 2, 1], activation=tf.nn.relu, scope='conv_3')(pool2)
        pool3 = conv3
        
        unfold = Unfold(scope='unfold')(pool3)
#         fc4 = FullyConnected(128, activation=tf.nn.relu, scope='fc4')(unfold)
        fc4 = unfold
        encoded = FullyConnected(20, activation=tf.nn.relu, scope='encode')(fc4)
#         pass 10000, training loss 1393.703857421875 # trained:1000000, 5059.127015 sec 5 feature
#         pass 10000, training loss 1324.6783447265625 trained:1000000, 1733.352743 sec 5feature 
    # pass 10000, training loss 993.19921875 trained:1000000, 4314.145067 sec 10
#     pass 10000, training loss 676.6449584960938 # trained:1000000, 1598.109703 20
        # decode
#         dfc4 = FullyConnected(128, activation=tf.nn.relu, scope='dfc4')(encoded)
        dfc4 = encoded
        decoded = FullyConnected(4*4*32, activation=tf.nn.relu, scope='decode')(dfc4)
        fold = Fold([-1, 4, 4, 32], scope='fold')(decoded)
        
        unpool3 = fold
        deconv3 = DeConvolution2D([3, 3, 32, 32], strides=[1, 2, 2, 1], output_shape=tf.shape(pool2), activation=tf.nn.relu, scope='deconv_3')(unpool3)
#         unpool3 = UnPooling((2, 2), output_shape=tf.shape(conv3), scope='unpool_3')(fold)
#         deconv3 = DeConvolution2D([3, 3, 32, 32], output_shape=tf.shape(pool2), activation=tf.nn.relu, scope='deconv_3')(unpool3)
        unpool2 = UnPooling((2, 2), output_shape=tf.shape(conv2), scope='unpool_2')(deconv3)
        deconv2 = DeConvolution2D([3, 3, 32, 32], output_shape=tf.shape(pool1), activation=tf.nn.relu, scope='deconv_2')(unpool2)
        unpool1 = UnPooling((2, 2), output_shape=tf.shape(conv1), scope='unpool_2')(deconv2)
        reconstruction = DeConvolution2D([3, 3, 1, 32], output_shape=tf.shape(x), activation=tf.nn.sigmoid, scope='deconv_1')(unpool1)
        # loss function
        loss = tf.nn.l2_loss(x - reconstruction)  # L2 loss

        # training
        training = tf.train.AdamOptimizer(1e-4).minimize(loss)

        #
        self.x = x
        self.reconstruction = reconstruction
        self.loss = loss
        self.training = training
        self.encoded = encoded
#         self.dfc4 = dfc4

    def train(self, batch_size, passes, new_training=True):
        """

        :param batch_size:
        :param passes:
        :param new_training:
        :return:
        """
        mnist = MNIST()

        with tf.Session() as sess:
            # prepare session
            if new_training:
                saver, global_step = Model.start_new_session(sess)
            else:
                saver, global_step = Model.continue_previous_session(sess, ckpt_file='saver/checkpoint')

            # start training, 需要修改数据获取的方法
            start = time.time()
            for step in range(1+global_step, 1+passes+global_step):
                x, y = mnist.get_batch(batch_size)
                self.training.run(feed_dict={self.x: x})

                if step % 10 == 0:
                    loss = self.loss.eval(feed_dict={self.x: x})
                    print("pass {}, training loss {}".format(step, loss))

                if step % 100 == 0:  
                    print('trained:%d, %f sec' % ((step-global_step)*batch_size, time.time() - start))
                
                if step % 1000 == 0:  # save weights
                    saver.save(sess, 'saver/cnn', global_step=step)
                    print('checkpoint saved')

    def weights_to_grid(self, weights, rows, cols):
        """convert the weights tensor into a grid for visualization"""
        height, width, in_channel, out_channel = weights.shape
        padded = np.pad(weights, [(1, 1), (1, 1), (0, 0), (0, rows * cols - out_channel)],
                            mode='constant', constant_values=0)
        transposed = padded.transpose((3, 1, 0, 2))
        reshaped = transposed.reshape((rows, -1))
        grid_rows = [row.reshape((-1, height + 2, in_channel)).transpose((1, 0, 2)) for row in reshaped]
        grid = np.concatenate(grid_rows, axis=0)
        return grid.squeeze()
                    
    def reconstruct(self):
        """

        """
        mnist = MNIST()

        with tf.Session() as sess:
            saver, global_step = Model.continue_previous_session(sess, ckpt_file='saver/checkpoint')

            # visualize weights
            first_layer_weights = tf.get_default_graph().get_tensor_by_name("conv_1/kernel:0").eval()
            grid_image = self.weights_to_grid(first_layer_weights, 4, 8)

            fig, ax0 = plt.subplots(ncols=1, figsize=(8, 4))
            ax0.imshow(grid_image, cmap=plt.cm.gray, interpolation='nearest')
            ax0.set_title('first conv layers weights')
            plt.show()

            # visualize results
            batch_size = 36
            x, y = mnist.get_batch(batch_size, dataset='testing')
            org, recon = sess.run((self.x, self.reconstruction), feed_dict={self.x: x})

            input_images = self.weights_to_grid(org.transpose((1, 2, 3, 0)), 6, 6)
            recon_images = self.weights_to_grid(recon.transpose((1, 2, 3, 0)), 6, 6)

            fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(10, 5))
            ax0.imshow(input_images, cmap=plt.cm.gray, interpolation='nearest')
            ax0.set_title('input images')
            ax1.imshow(recon_images, cmap=plt.cm.gray, interpolation='nearest')
            ax1.set_title('reconstructed images')
            plt.show()
            
    def encode_all(self):
        """

        """
        mnist = MNIST()

        with tf.Session() as sess:
            saver, global_step = Model.continue_previous_session(sess, ckpt_file='saver/checkpoint')

            codes = np.empty([50000,20], dtype = np.float32)
            labels = np.empty([50000,1], dtype = np.int)
            batch_size = 100
            start = time.time()
            for i in range(500):
                x, y = mnist.get_batch(batch_size, dataset='training')
                codes[i*batch_size:(i+1)*batch_size,] = sess.run((self.encoded), feed_dict={self.x: x}) 
                labels[i*batch_size:(i+1)*batch_size,] = y@(np.arange(10).reshape(10,1))
                if (i+1) % 100 == 0:
                    print('encoded:%d, %f sec' % ((i+1)*batch_size, time.time() - start))
                
            return codes,labels

    def encode(self, images):
        """

        """
        with tf.Session() as sess:
            saver, global_step = Model.continue_previous_session(sess, ckpt_file='saver/checkpoint')
            
#             codes = np.empty([images.shape[0],20], dtype = np.float32)
            codes = sess.run((self.encoded), feed_dict={self.x: images})
#             labels = np.empty([50000,1], dtype = np.int)
#             batch_size = 100
#             start = time.time()
#             for i in range(500):
#                 x, y = mnist.get_batch(batch_size, dataset='training')
#                 codes[i*batch_size:(i+1)*batch_size,] = sess.run((self.encoded), feed_dict={self.x: x}) 
#                 labels[i*batch_size:(i+1)*batch_size,] = y@(np.arange(10).reshape(10,1))
#                 if (i+1) % 100 == 0:
#                     print('encoded:%d, %f sec' % ((i+1)*batch_size, time.time() - start))
                
        return codes


    def decode(self, codes):
        with tf.Session() as sess:
            saver, global_step = Model.continue_previous_session(sess, ckpt_file='saver/checkpoint')
            output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['deconv_1/Sigmoid'])

        input_node_names_list = ['encode/Relu']
        inputs_replaced_graph_def = tf.GraphDef()
        for node in output_graph_def.node:
            if node.name in input_node_names_list:
                placeholder_node = tf.NodeDef()
                placeholder_node.op = "Placeholder"
                placeholder_node.name = node.name
                placeholder_node.attr["dtype"].CopyFrom(
                    tf.AttrValue(type=tf.float32.as_datatype_enum))
                inputs_replaced_graph_def.node.extend([placeholder_node])
            else:
                other_node = tf.NodeDef()
                other_node.CopyFrom(node)
                inputs_replaced_graph_def.node.extend([other_node])

        decode_graph_def = graph_util.extract_sub_graph(inputs_replaced_graph_def, ['deconv_1/Sigmoid'])
        decode_graph = tf.Graph()
        
        with decode_graph.as_default():
            input_x,input_y,result = tf.import_graph_def(decode_graph_def, return_elements=["encode/Relu:0","Placeholder:0","deconv_1/Sigmoid:0"])

            n = codes.shape[0]
            if n>100: n=100 
            fills = np.empty([n,28,28,1], dtype=np.float32)
                
            with tf.Session() as sess:
                img = sess.run(result,feed_dict={input_x:codes[0:n,:],input_y:fills})
                    
        num_grids = int(np.ceil(np.sqrt(n)))
        images = self.weights_to_grid(img.transpose((1, 2, 3, 0)), num_grids, num_grids)

        fig, ax = plt.subplots(ncols=1, figsize=(12, 12))
#         ax.imshow(images, cmap=plt.cm.gray, interpolation='nearest')
        ax.imshow(images, interpolation='nearest')
        plt.show()

            

In [ ]:
conv_autoencoder = ConvolutionalAutoencoder()

In [ ]:
# conv_autoencoder.train(batch_size=100, passes=10000, new_training=True)

In [ ]:
conv_autoencoder.reconstruct()

In [ ]:
codes,labels = conv_autoencoder.encode_all()

In [ ]:
mnist = MNIST()
x, y = mnist.get_batch(100, dataset='training')
code_test = conv_autoencoder.encode(x) 
label_test = y@(np.arange(10).reshape(10,1))
# print(code_test,label_test)
# ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan'] 
axis_x = np.arange(20)
fig, ax = plt.subplots(figsize = (16,12))
for i in range(code_test.shape[0]):
    ax.plot(axis_x,code_test[i],c=colors[int(label_test[i,0])])
plt.show()

In [ ]:
test_image = np.zeros([1,28,28,1])
code_test = conv_autoencoder.encode(test_image) 
code_test

In [ ]:
conv_autoencoder.decode(codes[0:100,])

In [ ]:
#20个特征，前10*5种，后10*5种
features = np.concatenate([20*np.eye(20), 60*np.eye(20), 100*np.eye(20), 140*np.eye(20), 180*np.eye(20)])
features.shape

In [ ]:
#20个特征，前10*5种，后10*5种
features = np.concatenate([20*np.eye(20)[0:10,:], 60*np.eye(20)[0:10,:], 
                           100*np.eye(20)[0:10,:], 140*np.eye(20)[0:10,:], 180*np.eye(20)[0:10,:],
                          20*np.eye(20)[10:20,:], 60*np.eye(20)[10:20,:], 
                           100*np.eye(20)[10:20,:], 140*np.eye(20)[10:20,:], 180*np.eye(20)[10:20,:]])
features.shape

In [ ]:
conv_autoencoder.decode(features)

In [ ]:
conv_autoencoder.decode(code_test)

In [ ]:
from sklearn.decomposition import PCA
PCA_2D = PCA(n_components=2)#2主成分

In [ ]:
minist_2D = PCA_2D.fit_transform(codes)#投影到2主成分空间
minist_2D.shape

In [ ]:
fig, ax = plt.subplots()
cax = ax.scatter(minist_2D[0:50000,0], minist_2D[0:50000,1], c=labels[0:50000,0], alpha=0.5)# 
fig.colorbar(cax)
ax.set_xlabel(r'$\Delta_i$', fontsize=15)
ax.set_ylabel(r'$\Delta_{i+1}$', fontsize=15)
ax.set_title('Volume and percent change')

ax.grid(True)
fig.tight_layout()

plt.show()

In [ ]:
from sklearn.manifold import TSNE
start = time.time()
model = TSNE(learning_rate=100, n_components=2, random_state=0, perplexity=5)
tsne5 = model.fit_transform(codes)
print('tsne5 finish，{}'.format(time.time() - start))

start = time.time()
model = TSNE(learning_rate=100, n_components=2, random_state=0, perplexity=30)
tsne30 = model.fit_transform(codes)
print('tsne30 finish，{}'.format(time.time() - start))

start = time.time()
model = TSNE(learning_rate=100, n_components=2, random_state=0, perplexity=50)
tsne50 = model.fit_transform(codes)
print('tsne50 finish，{}'.format(time.time() - start))

In [ ]:
plt.figure(1,figsize=(24, 48))
plt.subplot(311)
plt.scatter(tsne5[:, 0], tsne5[:, 1], c=labels[:,0], alpha=0.5)

plt.subplot(312)
plt.scatter(tsne30[:, 0], tsne30[:, 1], c=labels[:,0], alpha=0.5)

plt.subplot(313)
plt.scatter(tsne50[:, 0], tsne50[:, 1], c=labels[:,0], alpha=0.5)

plt.show()